# Orchestrator Jupyter Simple Test

In [1]:
# import the orch library in the local notebook env

import sys
import os
sys.path.insert(0, os.path.abspath('../../lib/python'))

In [2]:
import orch
import itertools 
import time

from promise import Promise

# Asyncronous Tasks

## functions

In [ ]:

@orch.Async
def task_1(arg):
    from random import randint
    import time
    
    sleep = randint(1,10)
    sleep = arg
    print("task 1 with arg: %s sleep %d" % (arg, sleep))
    time.sleep(sleep)
    print("task 1 with arg: %s done sleep %d" % (arg, sleep))

    return "task_1(%s)" % arg 

@orch.Async
def task_2(pair):
    import sys 

    print("task_2 with args:", pair)

    return "%s+%s" % (pair[0],pair[1])

@orch.Async
def joiner(lst,n):
    print("joiner waiting for ",lst)
    resolved_lst = [ x.get() for x in lst ]
    print("joiner ",resolved_lst)
    l = list(zip(*[itertools.islice(resolved_lst, i, None, n) for i in range(n)]))
    r = [ (a,b) for a,b in l]
    print("joined: ", r)
    return r

## Local workflow with async calls only

In [ ]:
do_local = True

args=[1,2,3,4,5,6,7,8,9,10,11,12]

if do_local:
    # wrapping the function in an asyncronous call
    @orch.Async
    def process_arg(arg):
        p = task_1(arg)
        return p

    res = []
    for arg in args:
        # run each task_1 as a promise 
        p = Promise.resolve(process_arg(arg))
        res.append(p)

    # get all the promise responses and *then*, the join each output in tuples and *then*, call task_2
    p = Promise.all(res).then(lambda x: joiner(x,2) ).then(lambda x : [task_2(p) for p in x.get()]).get()

    print("final promise: ",p)
    
    result = [ x.get() for x in p ]
    
    print(result)

    print("Local Tasks example done")

## Remote Tasks wrapped manually in Jobs example with barrier on the first task

In [ ]:
args=[1,2,3,4,5,6,7,8,9,10,11,12]

def job_1(arg):
    job=orch.Job(params={\
        "ntasks":"1",
        "nodes":"1",
        "job-name":"task_1",
        "cpus-per-task":"1"
    })
    job.setVerbose(True)
    ret = job.run(task_1, arg)
    return(ret)

@orch.Async
def job_2(arg):
    job=orch.Job(params={\
        "ntasks":"1",
        "nodes":"1",
        "job-name":"task_2",
        "cpus-per-task":"1",
    })
    job.setVerbose(True)
    ret = job.run(task_2, arg)
    return(ret)

args=[1,2,3,4,5,6,7,8,9,10,11,12]

do_example_2 = True

if do_example_2:
    res = []
    
    for arg in args:
        res.append(Promise.resolve(job_1(arg)))

    results = Promise.all(res).then(lambda x: joiner(x,2) ).then(lambda x : [job_2(p) for p in x.get() ]).get()


In [ ]:
print(result)

In [ ]:
if do_example_2:
    print("results: ",results)
    
    resolved_result = [x.get() for x in results]
    print("resolved result: ",resolved_result)
    
    print("example 2 done")

## Remote tasks with parallel workflows

In [ ]:

args=[[1,2,3,4],[5,6,7,8],[9,10,11,12]]

def job_t1(arg):
    job=orch.Job(params={\
        "ntasks":"1",
        "nodes":"1",
        "job-name":"task_1",
        "cpus-per-task":"1"
    })
    print("job_t1 starting for ",arg)
    job.setVerbose(True)
    ret = job.run(task_1, arg)
    return(ret)

@orch.Async
def joiner(lst,n):
    print("joiner waiting for ",lst)
    resolved_lst = [ x.get() for x in lst ]
    print("joiner ",resolved_lst)
    l = list(zip(*[itertools.islice(resolved_lst, i, None, n) for i in range(n)]))
    return [ (a,b) for a,b in l ]

@orch.Async
def job_t2(arg):
    arg = arg.get()
    print("job_t2: ", arg)
    job=orch.Job(params={\
        "ntasks":"1",
        "nodes":"1",
        "job-name":"task_2",
        "cpus-per-task":"1"
    })
    print("job_t2 starting for ",arg)
    job.setVerbose(True)
    ret = job.run(task_2, arg)
    return(ret)

do_example_3 = True
if do_example_3:
    def process_list(partial_list):
        stage_1 = []
        for data in partial_list:
            stage_1.append(Promise.resolve(job_t1(data)))  
        return Promise.all(stage_1).then(lambda x: joiner(x,2)).then(lambda x: job_t2(x)).get()

    # start processing the partial lists in args
    results = [ process_list(x) for x in args ]

In [ ]:
if do_example_3:
    print("result: ",results)

    resolved_result = [x.get() for x in results]
    print("resolved result: ",resolved_result)
    print("Done")

# Functions as Jobs

## two serial jobs called programatically

In [3]:
@orch.asJob(cores=1)
def job_1(arg1, arg2):
    print("Job 1:",arg1,arg2)
    return (arg1,arg2)
    
@orch.asJob(cores=1)
def job_2(arg1, arg2):
    print("Job 2:",arg1,arg2)
    return (arg1,arg2)

response_j1 = job_1("a","b")

# the return type is a future object wrapped as a ProcessAsyncCall
print(response_j1)

# get is a blocking function to get the results from job_1
r1,r2 = response_j1.get()

# after job_1 resolved, calling job_2 as a blocking call
r3,r4 = job_2(r1,r2).get()

print(r3,r3)

Job 1: a b
Job 2: a b
a a


## two serial jobs called via then

In [4]:
@orch.asJob(cores=1)
def job_1(arg1, arg2):
    print("Job 1:",arg1,arg2)
    return (arg1,arg2)
    
@orch.asJob(cores=1)
def job_2(a1,a2):
    
    print("Job 2:",a1,a2)
    return (a1,a2)

a,b = job_1("a","b").then(job_2).get()
print(a,b)

Job 1: a b
Job 2: a b
a b
